# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

**Connect with an LLM and start a conversation with it**

* For this project, we will use OpenAI's gpt-3.5-turbo

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

**Call the ChatModel (the LLM)**

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and peaceful color. It's no wonder it's your favorite! What do you like most about the color blue?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 13, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-98178d80-cbce-4142-aeb0-c5400efbbd1e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 28, 'total_tokens': 41})

 **Check if the Chatbot remembers your favorite color.**

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have access to personal information about you, so I do not know what your favorite color is. Can you please tell me?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ebd2d2f-e326-4eaa-931a-49eeb0cc24bc-0', usage_metadata={'input_tokens': 13, 'output_tokens': 32, 'total_tokens': 45})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

In [10]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [11]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

FileChatMessageHistory will be create a new file with messages

memory_key and variable_content should have the same name

In [12]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [13]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [14]:
chain.invoke("Hello!")

{'content': 'Hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [15]:
chain.invoke("my name is Marcela")

{'content': 'my name is Marcela',
 'messages': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Nice to meet you, Marcela! How can I help you today?'}

In [16]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='Hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is Marcela'),
  AIMessage(content='Nice to meet you, Marcela! How can I help you today?')],
 'text': 'Your name is Marcela.'}

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using chatGPT API.